<h1>
    Imports

In [4]:
import numpy as np
import random

<h1>
    Loading Data
   

In [5]:
X_train = np.genfromtxt('X_train.csv', delimiter = ",")
y_train = np.genfromtxt('y_train.csv')
X_test = np.genfromtxt('X_test.csv', delimiter = ",")

lambda_input = int(random.randrange(2**15)%10)
sigma2_input = float(random.randrange(2**15)%10 + 1)


<h1>
Part 1

PART 1: In this part you will implement the $\ell_2$-regularized least squares linear regression algorithm we have been discussing (ridge regression). Recall from the lectures that this takes the form:

$$w_{RR} = \arg\min_w \|y - Xw\|^2 + \lambda\|w\|^2.$$

Your task will be to write code that takes in data $y$ and $X$ and outputs $w_{RR}$ for an arbitrary value of $\lambda$.

In [6]:
## Solution for Part 1
def part1(lambda_input,X_train,y_train):
    ## Input : Arguments to the function
    ## Return : wRR, Final list of values to write in the file
    
    # first storing X^T.X + lambda
    d = X_train.shape[1]
    temp = lambda_input*np.eye(d) + X_train.T.dot(X_train)
    WRR = (np.linalg.inv(temp)).dot(X_train.T.dot(y_train))
    return WRR

In [7]:
wRR = part1(lambda_input,X_train,y_train)
print(wRR)

[-4.58812731e-01  5.63736678e-02 -5.75872197e-01 -4.73589093e+00
  1.12243202e-01  2.68725316e+00  1.15241426e-03]


<h1>
    Part 2

In the same code, you will also implement the active learning procedure discussed in Lecture 5. For this problem, we will provide you with an arbitrary setting of $\lambda$ and $\sigma^2$ and ask you to provide us with the first 10 locations you would measure from a set $\mathcal{D} = \{x\}$ given a set of measured pairs (y, X). Please look over the slides carefully to remind yourself about the sequential evolution of the sets $\mathcal{D}$ and (y,X).

In [8]:
def update(lambda_input,sigma2_input,X_train,d,y_train,old_xx,old_xy):
    old_xx = X_train.T.dot(X_train)
    old_xy = X_train.T.dot(y_train)
    new_var_inv = lambda_input * np.eye(d) + (1/sigma2_input)*old_xx
    new_variance = np.linalg.inv(new_var_inv)
    new_mean_temp = (lambda_input*sigma2_input*np.eye(d) + old_xx)
    new_mean = np.linalg.inv(new_mean_temp).dot(old_xy)
    return new_variance,new_mean,old_xx,old_xy


In [13]:
## Solution for Part 2
def part2(lambda_input,sigma2_input,X_train,X_test,y_train):
    ## Input : Arguments to the function
    ## Return : active, Final list of values to write in the file
    d = X_train.shape[1]
    active = []
    old_xx = np.zeros((d,d))
    old_xy = np.zeros((d))
    new_var,new_mean,old_xx,old_xy = update(lambda_input,sigma2_input,X_train,d,y_train,old_xx,old_xy)
    wRR = new_mean
    indices = list(range(X_test.shape[0]))
    for i in range (0,10):
        var_matrix = (X_test.dot(new_var)).dot(X_test.T)
        row_largest = np.argmax(var_matrix.diagonal())
        X_train = X_test[row_largest,:]
        y_train = X_train.dot(wRR)
        actual_row = indices[row_largest]
        active.append(actual_row)
        X_test = np.delete(X_test, row_largest, axis=0)
        indices.pop(row_largest)
        new_var, new_mean, old_xx, old_xy = update(lambda_input, sigma2_input, X_train, d, y_train, old_xx, old_xy)
        wRR = new_mean
    active = [i + 1 for i in active]
    return active


In [14]:
active = part2(lambda_input,sigma2_input,X_train,X_test,y_train)
active

[25, 31, 16, 32, 24, 21, 2, 27, 7, 9]